In [4]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib import request

In [3]:
URL: str = "https://www.bot.jfn.ac.lk/index.php/research-grants/"

In [45]:
with request.urlopen(URL) as conn:
    html: str = conn.read()

In [46]:
soup = BeautifulSoup(html)

In [47]:
grants = soup.find("div", attrs = {"class": "entry clr", "itemprop": "text"})  # <div class="entry clr" itemprop="text">

In [48]:
# grants grouped, year wise.

grant_groups = grants.find_all("div", attrs = {"class": "su-spoiler su-spoiler-style-fancy su-spoiler-icon-plus su-spoiler-closed"})

In [49]:
grant_groups[3]

<div class="su-spoiler su-spoiler-style-fancy su-spoiler-icon-plus su-spoiler-closed" data-anchor-in-url="no" data-scroll-offset="0"><div class="su-spoiler-title" role="button" tabindex="0"><span class="su-spoiler-icon"></span>2017</div><div class="su-spoiler-content su-u-clearfix su-u-trim">
<table class="tablepress tablepress-id-RG2017" id="tablepress-RG2017">
<thead>
<tr class="row-1 odd">
<th class="column-1">Grantee</th><th class="column-2">Project Title</th><th class="column-3">Funding Source</th><th class="column-4">Amount</th>
</tr>
</thead>
<tbody class="row-hover">
<tr class="row-2 even">
<td class="column-1">Prof.R.Kapilan</td><td class="column-2"></td><td class="column-3">University research  grant</td><td class="column-4">LKR 800,000.00</td>
</tr>
<tr class="row-3 odd">
<td class="column-1">Dr. E.C. Jeyaseelan</td><td class="column-2">Molecular identification of causative agent of a new wide spreading disease on Okra in Northern Sri Lanka.</td><td class="column-3">Universi

In [61]:
table = {"Year": [], "Grantee": [], "Project": [], "Funder": [], "Amount": []}

for group in grant_groups:  # grants grouped by year
    year: str = group.find("div", attrs = {"class": "su-spoiler-title"}).text
    if not year:
        raise RuntimeError
        
    for grant in group.find_all("tr")[1:]: # for each grant in the select year, skip the title row
        grantee: str = grant.find("td", attrs = {"class": "column-1"}).text
        project: str = grant.find("td", attrs = {"class": "column-2"}).text
        funder: str = grant.find("td", attrs = {"class": "column-3"}).text
        amount: str = grant.find("td", attrs = {"class": "column-4"}).text
        
        if (grantee):
            table["Year"].append(year)
            table["Grantee"].append(grantee)
            table["Project"].append(project)
            table["Funder"].append(funder)
            table["Amount"].append(amount)

In [63]:
data = pd.DataFrame(table)

In [80]:
data.Amount = data.Amount.str.replace("LKR", "").str.strip()
data.Grantee = data.Grantee.str.replace('\n', ' ')
data.Funder = data.Funder.str.replace('\t', '').str.title()
data.Project = data.Project.str.title()

In [86]:
# commas will cause trouble in csv parsing
data.Amount = data.Amount.str.replace(',', '').str.replace(' ', '')

In [88]:
data.to_csv("./grants.csv", index = None)

In [89]:
# woohoo :))